# [196. Delete Duplicate Emails](https://leetcode.com/problems/delete-duplicate-emails/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Person

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| email       | varchar |
+-------------+---------+</pre>
id is the primary key (column with unique values) for this table.
Each row of this table contains an email. The emails will not contain uppercase letters.
 

Write a solution to delete all duplicate emails, keeping only one unique email with the smallest id.

For SQL users, please note that you are supposed to write a DELETE statement and not a SELECT one.

For Pandas users, please note that you are supposed to modify Person in place.

After running your script, the answer shown is the Person table. The driver will first compile and run your piece of code and then show the Person table. The final order of the Person table does not matter.

The result format is in the following example.

 

Example 1:

Input: 
Person table:
<pre>+----+------------------+
| id | email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
| 3  | john@example.com |
+----+------------------+</pre>
Output: 
<pre>+----+------------------+
| id | email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
+----+------------------+</pre>
Explanation: john@example.com is repeated two times. We keep the row with the smallest Id = 1.

In [1]:
# pandas schema

import pandas as pd

data = [[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id': 'int64', 'email': 'object'})

In [2]:
# to spark dataframe

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

person_df = spark.createDataFrame(person)
person_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/11 19:58:07 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/11 19:58:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/11 19:58:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+----------------+
|id |email           |
+---+----------------+
|1  |john@example.com|
|2  |bob@example.com |
|3  |john@example.com|
+---+----------------+


In [3]:
from pyspark.sql import Window

# solving in spark dataframe

# spark does not support direct delete being OLAP system.
# One solution is to re-write the data after applying filter. (not done in this example)

person_df \
    .withColumn('rn', F.row_number().over(Window.partitionBy('email').orderBy('id'))) \
    .filter(F.col('rn') == 1) \
    .drop('rn') \
    .show()

+---+----------------+
| id|           email|
+---+----------------+
|  2| bob@example.com|
|  1|john@example.com|
+---+----------------+


In [4]:
# solving in spark SQL

person_df.createOrReplaceTempView('person')

spark.sql('''
select 
    id, email 
from 
    (select *,row_number() over (partition by email order by id) as rn from person) p 
where p.rn=1;
''').show()

+---+----------------+
| id|           email|
+---+----------------+
|  2| bob@example.com|
|  1|john@example.com|
+---+----------------+


In [5]:
# mysql solution for OLTP:
'''
delete
from person 
where id in (
    select 
        id 
    from 
        (select id,row_number() over (partition by email order by id) as rn from person) p 
    where p.rn>1
 );
'''

'\ndelete\nfrom person \nwhere id in (\n    select \n        id \n    from \n        (select id,row_number() over (partition by email order by id) as rn from person) p \n    where p.rn>1\n );\n'

In [6]:
spark.stop()